## 9.4  Brute Force Calculation of a Root

This is complete code and output for you to review.  Note the following in your review.
<ul>
    <li>assertions are used to check the input</li>
    <li>the output is displayed with 17 decimal digits even though double precision floating point values are only good to 15-16 decimal digits; this is to better display the chaotic effects of floating point error</li>
        <li>the outer loop (i) loops over each successive decimal digit.  Consider an appoximation to a root written as (integer i_lo).d<sub>1</sub>d<sub>2</sub>...d<sub>i</sub>...d<sub>decimals</sub>.  For example, 99.1234567 where i_lo = 99, d<sub>1</sub> = 1, d<sub>2</sub> = 2, etc.  `i` = 4 indicates the 4th digit after the decimal point in the `decimals` = 7 digits in the approximation.</li>
    <li>the inner loop (j) looks sequentially through 1, 2, 3... for the least upper bound of the interval where the function changes sign.  For example, if the lower bound on the root is currently x_lo = 1.25, then decimal digit d<sub>3</sub> will be set to j=1 (x = 1.251) for interval &#91;1.250, 1.251&#93;, then j=2 (x = 1.252) for interval &#91;1.251, 1.252&#93;, j=3 (1.253),... until the interval where the function changes sign is found</li>
    <li>if f(x<sub>j-1</sub>) and f(x<sub>j</sub>) have opposite signs such that their product is negative then the function has crossed the x axis and there is a root somewhere between x<sub>j-1</sub> and x<sub>j</sub></li>
   <li>the break statement is used to terminate the inner loop when the function changes sign by crossing the x axis</li>
   </ul>
<br />
Carefully consider the example problem, which requests decimals=18 decimal digits of precision while the output stops at 16 decimal digits.  This is because there are no floating point numbers between the lower and upper bounds computed to 16 decimal digits:  the two numbers are equal (as you can see in the output in the 16 decimals row).  Thus, the statement `if (f*f_lo &lt; 0.0)` &mdash; which leads to displaying output &mdash; can never again be true since anything times itself will always be non-negative.  This means that the loops continue through their computations to their ends but with no more output.  If we had requested a trillion decimals and the `assert decimals <= 20` statement wasn't there, the code would happily keep computing for a long time after appearing to stop at 16 decimal digits. Be aware of this type of insidious danger when you code a loop.<br />
<br />
Finally, observe the output to 17 digits for the `sample function x**3 - 2`.  Only the values 1.000..., 2.000..., and 1.250... appear to be "exact" to all 17 digits.  This is because these numbers are exactly representable in binary.  In particular, the decimal fraction 0.25 is given by 0 &ast; 2<sup>-1</sup> + 1 &ast; 2<sup>-2</sup> + 0 &ast; 2<sup>-3</sup> such that the significand is 0100....0000<sub>2</sub>.

In [ ]:
def bound_root(func, i_lo, decimals = 10):
    '''iteratively estimates root of `func` above integer `i_lo` to `decimals` digits
    
    Input
       func     - function to plot
       i_lo     - greatest integer less than the root
       decimals - number of decimals of precision to iterate to
    Output
       None; print sequence of lower and upper bounds precise to `decimals` decimal digits
                (precise to within floating point error, that is)
             and plot the convergence of the bounds
             
    Some input data error checking is performed using assert statements.
    '''
    #######################
    # check input data
    #
    assert decimals <= 20, "do not waste time with decimals > 20; double precision is at best ~15-16 decimal digits"
    
    assert type(i_lo) == int, "i_lo must be an integer"   # look up the Python `type` function

    f_lo = func(float(i_lo))
    f_hi = func(float(i_lo+1))
    assert f_lo*f_hi < 0.0, "There should be exactly one root in [i_lo, i_lo+1]"
    #
    #######################

    import numpy as np
    L = np.zeros(decimals+1)           # set up an array of lower bounds to display
    U = np.zeros(decimals+1)           # set up an array of upper bounds to display
    L[:] = np.nan                      # initialize the array elements to NaN -- `Not a Number`
    U[:] = np.nan                      # initialize the array elements to NaN -- `Not a Number`
    L[0] = float(i_lo)                 # the first element of the lower bound array is the integer input
    U[0] = float(i_lo+1)               # the first element of the upper bound array is the next higher integer

    print()
    print("decimals  L[i]                  U[i]")
    print("-------- --------------------- ---------------------")
    string  = '{:5d}'.format(0) + "  "
    string += '{:22.17f}'.format(L[0])
    string += '{:22.17f}'.format(U[0])
    print(string)

    x_lo = float(i_lo)                 # initialize x_lo to the input lower bound integer
    f_lo = func(x_lo)                  # compute the function at this lower bound

    for i in range(1, decimals+1):     # iterate through each successive decimal digit d_1, d_2,... d_decimals
        fraction = 10**(-i)            # compute the magnitude of that decimal place
        for j in range(1, 11):         # loop through digits 1-9 with the upper bound possibly at 10
            x = x_lo + j*fraction      # x_lo has i decimal digits; sequence through the j more precise decimal places
            f = func(x)                # compute the function at this new x value
            if (f*f_lo < 0.0):         # if a root is detected because the function changed sign...
                U[i] = x               #    then the upper bound (above the root) is this x value
                L[i] = x - fraction    #    and the lower bound (below the root) was the value just before
                string  = '{:5d}'.format(i) + "  "
                string += '{:22.17f}'.format(L[i])
                string += '{:22.17f}'.format(U[i])
                print(string)
                x_lo = L[i]            # set x_lo to the lower bound and
                f_lo = func(x_lo)      # compute the function here
                break                  # then break from the j loop and go to the next more precise decimal digit i

    # find the last valid estimate of the root in L
    for i in range(decimals,0,-1):
        if not np.isnan(L[i]):         # look for last element that is a number (and not a NaN)
            end = i
            break
    # plot the convergence of the two bounds to the root
    import matplotlib.pyplot as plt
    fig = plt.figure(1)
    plt.yscale('log')
    plt.plot(range(0,end), (U[0:end] - L[0:end])/2.0, 'r-', linewidth = 2.0)
    plt.xlabel('outer loop iteration', fontsize=12, labelpad=10)
    plt.ylabel('uncertainty in root estimate:  (U-L)/2', fontsize=12, labelpad=10)
    title_string  = "convergence of outer loop for brute force calculation of root\n"
    title_string += func.__doc__
    plt.title(title_string)
    plt.show()

    return None

def func(x):
    '''sample function x**3 - 2'''
    f = x**3 - 2.0
    return f
i_lo = 1
bound_root(func, i_lo, decimals = 18)
